In [7]:
import cohere
import os
import numpy as np
import math
import pickle


# import openai_secret_manager
import openai
import re
import json
from dotenv import load_dotenv

load_dotenv()
key = os.getenv('cohere_key')
co = cohere.Client(key)
openai.api_key =  os.getenv('openai_key')


In [8]:
file = open("C:\\Users\\bouta\\OneDrive\\Bureau\\hackathons\\test\\data\\data.txt", "r")

texts = []
for line in file:
    texts.append(line.split(":")[1])

myRecipesFromData = texts

In [9]:
response = co.embed(
    texts=texts,
    model='small',
)
embeddings=response.embeddings
all_embeddings = np.array(embeddings)
np.save('embeddings.npy', all_embeddings)


In [10]:
""" let's test here first """


response = co.embed(
    texts=["ginger, Lentils, tomatoes, onions, parsley, cilantro, vermicelli noodles, ginger, saffron, black pepper."],
    model='small',
)
embeddingForRecipeTest=response.embeddings

cosine_similarity_list = []
all_embeddings = np.load('embeddings.npy')
for i in range(len(all_embeddings)):
    # Calculate the dot product of the two arrays
    dot_product = np.dot(embeddingForRecipeTest, all_embeddings[i])

    # Calculate the norm of the two arrays
    norm_embeddingForRecipeTest = np.linalg.norm(embeddingForRecipeTest)
    norm_embedding = np.linalg.norm(all_embeddings[i])

    # Calculate the cosine similarity
    cosine_similarity = dot_product / (norm_embeddingForRecipeTest * norm_embedding)

    cosine_similarity_list.append((i, cosine_similarity))


# Sort the list by the variable in descending order
cosine_similarity_list.sort(key=lambda x: x[1], reverse=True)

# Select the top 3 by ID
top_3 = [x for x in cosine_similarity_list[:3]]
print("Top 3 IDs:", top_3)

# for recipe in top_3:
#     print(recipe[1], myRecipesFromData[recipe[0]])



Top 3 IDs: [(3, array([0.93595866])), (51, array([0.9127999])), (7, array([0.88192896]))]


In [11]:


# Authenticate with OpenAI API



# Define function to generate recipe suggestions
def generate_recipe_suggestions(ingredients, diet):
    if diet is not None:
        prompt = f"Generate a traditional Moroccan recipe using: {ingredients}. Be careful, The person has {diet}. For the response It should be a json object that contains, Name of the recipe, Ingredients, Instructions, Calories and the last one the date of today."
    else:
        prompt = f"Generate a traditional Moroccan recipe using: {ingredients}. For the response It should be a json object that contains, Name of the recipe, Ingredients, Instructions, Calories and the last one the date of today."
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )
    recipe = json.loads(json.dumps(response))["choices"][0]["text"]
    return recipe



diet = None

# Generate recipe suggestions and filter to only include traditional Moroccan recipes
recipe_suggestions = []
for recipe in top_3:
    generatedRecipe = generate_recipe_suggestions(myRecipesFromData[recipe[0]], diet)
    recipe_suggestions.append(json.loads(generatedRecipe))
    
print(recipe_suggestions)

[{'recipe': {'name': 'Moroccan Lentil Soup', 'ingredients': ['1 cup lentils', '1 cup chickpeas', '3 tomatoes', '1 onion', '1 celery', '1 parsley', '1 cilantro', '1 lemon juice', '1 vermicelli noodles', '1 olive oil', '1 cumin', '1 ginger', '1 cinnamon', '1 saffron', '1 salt', '1 black pepper'], 'instructions': 'Soak the lentils and chickpeas overnight. In a large pot, sauté the onions, celery, and garlic in olive oil until softened. Add the tomatoes and cook until they begin to break down. Add the lentils, chickpeas, and enough water to cover the ingredients. Bring to a boil and simmer for 30 minutes. Add the vermicelli noodles and cook for an additional 10 minutes. Season with salt, pepper, cumin, ginger, cinnamon, and saffron. Stir in the lemon juice and cilantro just before serving.', 'calories': '500', 'date': '4/21/2020'}}, {'Name': 'Traditional Moroccan Recipe', 'Ingredients': ['1 lb. lamb or beef', '1 onion', '4 cloves garlic', '1 celery stalk', '1/4 cup parsley', '1/4 cup cilan